 ## <center>TOP ACTION MOVIES</center>

using the famous rotten tomatoes website, we scrape the top 140 action movies and their details based on their rotten tomato score.

This web scraping is processed using Python packages, "requests" and "beautifulSoup"

### Step-1 Setup

In this step, we will setup our environment for scraping the website.

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Using rotten tomatoes website for top action movies to watch, and using the 'requests' package's '.get' method, we get the html file of the page

In [6]:
base_url='http://editorial.rottentomatoes.com/guide/140-essential-action-movies-to-watch-now/2/'
r = requests.get(base_url)
html=r.content
html[:5000]

b'<!DOCTYPE html>\n<html lang="en-US">\n    <head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#">\n    <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n        <meta property=\'og.description\' content="From John Wick and Die Hard to Mad Max and Atomic Blonde, these best action movies ever will thrill you and get the adrenaline pumping!" />\n    <meta name=\'description\' content="From John Wick and Die Hard to Mad Max and Atomic Blonde, these best action movies ever will thrill you and get the adrenaline pumping!" />\n    <meta property=\'og:title\' content="140 Essential Action Movies To Watch Now" />\n    <meta property=\'og:type\' content="article" />\n    <meta property=\'og:image\' content="https://s3-us-west-2.amazonaws.com/flx-editorial-wordpress/wp-content/uploads/2019/06/06180025/RT_140_ESSENTIAL_ACTION_600x314.jpg" />\n    <meta property=\'og:url\' content="https://editorial.rottentomatoes.com/guide/140-essentia

From the above output, it doesn't look understandable. so, to make it more understandable, we will use the BeautifulSoup module, to parse the html file.

BeautifulSoup has three parsers in total, and its documentation suggests, 'lxml' is their best parser

In [14]:
soup = BeautifulSoup(html, "lxml")


From above outputs, we can see that the parsed file is much more arranged and understandable for us to locate required classes and tags.

For crosschecking, we can write the parsed html to a file, and save on our local machine and check if it was parsed correctly.

Now, once we parse the html file, we need to find out which part of the file do we have the required information.
we need **Title, released year, Director, rotten tomatoes score, adjusted score, Synopsis, Description and Cast of the film**


Now, to find out which part of the html has these data, we need to use a brower, (I used Chrome) right click on the title and select 'Inspect' on the dropdown, which will give us the html page in the developers column.

Upon inspection, we found that all the related data is under the **<div class='col-sm-18 col-full-xs countdown-item-content'**

Using BeautifulSoups' find_all method we will retrive with that particular class name and tag from the html file

In [13]:
divs=soup.find_all('div',class_='col-sm-18 col-full-xs countdown-item-content')


### Step -2

In this step, we will filter and extract the required content from the html file we parsed

once we got retrieved the related content, we need to find the part where the movies details that we need are present.

Upon inspection we see that the **title, year and score** are present in the **href** with **<a** tag

Using list comprehension, and fin_all method we filter all the lines with **<a** tag and **h2** ref

In [17]:
headings=[div.find('h2') for div in divs]
type(headings)

list

We see that the is a list, so we take the first 3 elements of the list using indexing and study the contents of the elements


In [18]:
headings[0:4]

[<h2><a href="https://www.rottentomatoes.com/m/1018009-running_scared/">Running Scared</a> <span class="subtle start-year">(1986)</span> <span class="icon tiny fresh" title="Fresh"></span> <span class="tMeterScore">60%</span></h2>,
 <h2><a href="https://www.rottentomatoes.com/m/equilibrium/">Equilibrium</a> <span class="subtle start-year">(2002)</span> <span class="icon tiny rotten" title="Rotten"></span> <span class="tMeterScore">40%</span></h2>,
 <h2><a href="https://www.rottentomatoes.com/m/hero/">Hero</a> <span class="subtle start-year">(2004)</span> <span class="icon tiny certified" title="Certified Fresh"></span> <span class="tMeterScore">95%</span></h2>,
 <h2><a href="https://www.rottentomatoes.com/m/1017666-road_house/">Road House</a> <span class="subtle start-year">(1989)</span> <span class="icon tiny rotten" title="Rotten"></span> <span class="tMeterScore">39%</span></h2>]

once we filter all the **h2** elements from the html, we observer that

**title** is in **<a** tag,

**year** is in **<span** tag with **class='subtle start-year'**

**score** is in **<span** tag with **class='tMeterScore**

Using list comprehensions, we will filter the desired content

In [19]:
movie_names=[heading.find('a').string for heading in headings]
movie_names[:10]

['Running Scared',
 'Equilibrium',
 'Hero',
 'Road House',
 'Unstoppable',
 'Shaft',
 'The Villainess (Ak-Nyeo)',
 'Highlander',
 'Die Hard 2',
 'National Treasure']

In [20]:
movie_years=[year.find('span', class_='subtle start-year').string for year in headings]
movie_years[:10]

['(1986)',
 '(2002)',
 '(2004)',
 '(1989)',
 '(2010)',
 '(1971)',
 '(2017)',
 '(1986)',
 '(1990)',
 '(2004)']

we notice that the years have '(',')' around them and are of string type, we need to remove those brackets and convert them into int data type

In [22]:
years=[year.strip('()') for year in movie_years]
years= [int(year) for year in years]
years[:10]

[1986, 2002, 2004, 1989, 2010, 1971, 2017, 1986, 1990, 2004]

The scores have '%' and in the format string, so we will use strip() method to remove the % sign and convert them into interger

In [23]:
t_score=[heading.find('span',class_="tMeterScore").string for heading in headings]
original_scores=[int(x.strip('%')) for x in t_score]
original_scores[0:5]

[60, 40, 95, 39, 86]

Now that we got all the important data we needed from that particular class. Now we need **Consensus**. Again by inspecting the the html page, we find that consensus is in **'div tag** and  **class_='info critics-consensus**

Using BeasutifulSoup's find_all method we filter the class

In [24]:
consensus=[div.find('div', class_='info critics-consensus') for div in divs]
consensus[0]

<div class="info critics-consensus"><span class="descriptor">Critics Consensus:</span> Running Scared struggles to strike a consistent balance between violent action and humor, but the chemistry between its well-matched leads keeps things entertaining.</div>

Now, let's see what does the consensus have in it's content

In [25]:
for x in consensus[0]:
    print(x)

<span class="descriptor">Critics Consensus:</span>
 Running Scared struggles to strike a consistent balance between violent action and humor, but the chemistry between its well-matched leads keeps things entertaining.


So, the second part of the each **<div** tag has required consensus.and it is also observerd that there is an unwanted space presnet right before the consensus text. we will use the strip menthod to remove the space.

In [26]:
consensus[0].contents[1].strip()

'Running Scared struggles to strike a consistent balance between violent action and humor, but the chemistry between its well-matched leads keeps things entertaining.'

Now, we will apply this for all the elements consensus list. (using List comprehension)

In [27]:
consensus_text=[con.contents[1].strip() for con in consensus]
consensus_text[0:5]

['Running Scared struggles to strike a consistent balance between violent action and humor, but the chemistry between its well-matched leads keeps things entertaining.',
 'Equilibrium is a reheated mishmash of other sci-fi movies.',
 'With death-defying action sequences and epic historic sweep, Hero offers everything a martial arts fan could ask for.',
 "Whether Road House is simply bad or so bad it's good depends largely on the audience's fondness for Swayze -- and tolerance for violently cheesy action.",
 "As fast, loud, and relentless as the train at the center of the story, Unstoppable is perfect popcorn entertainment -- and director Tony Scott's best movie in years."]

Next is Directors. Again by inspecting the the html page, we find Director of each title is in **'div** tag and **class_='info director**

In [28]:
directors_list=[div.find('div', class_='info director') for div in divs]
directors_list[0:5]

[<div class="info director">
 <span class="descriptor">Directed By:</span> <a class="" href="/celebrity/peter_hyams/">Peter Hyams</a></div>,
 <div class="info director">
 <span class="descriptor">Directed By:</span> <a class="" href="/celebrity/kurt_wimmer/">Kurt Wimmer</a></div>,
 <div class="info director">
 <span class="descriptor">Directed By:</span> <a class="" href="/celebrity/zhang_yimou/">Zhang Yimou</a></div>,
 <div class="info director">
 <span class="descriptor">Directed By:</span> <a class="" href="/celebrity/rowdy_herrington/">Rowdy Herrington</a></div>,
 <div class="info director">
 <span class="descriptor">Directed By:</span> <a class="" href="/celebrity/tony_scott/">Tony Scott</a></div>]

Once, we filter the class that has Director, we observe that the name of the director is in tag **<a**.

Using list comprehension we filter the **<a** tag to get the director's name

In [29]:
directors=[x.find('a') for x in directors_list]
directors[:5]

[<a class="" href="/celebrity/peter_hyams/">Peter Hyams</a>,
 <a class="" href="/celebrity/kurt_wimmer/">Kurt Wimmer</a>,
 <a class="" href="/celebrity/zhang_yimou/">Zhang Yimou</a>,
 <a class="" href="/celebrity/rowdy_herrington/">Rowdy Herrington</a>,
 <a class="" href="/celebrity/tony_scott/">Tony Scott</a>]

from the output we observe thata, some of the director has 'None', and upon inspecting the website, it is indeed seen that some of the movies, doesn't have directors listed. so we need to fill those empty places with None for our list.

If we filter **<a** for the director names, we get an error because, we cannot filter none, hence we need to assign **None** in the place, with no director

In [30]:
directors=[None if x.find('a') is None else x.find('a').string for x in directors_list]
directors[0:5]

['Peter Hyams', 'Kurt Wimmer', 'Zhang Yimou', 'Rowdy Herrington', 'Tony Scott']

Similarly, we get the cast, and upon inspecting the website we find that the cast details are available in **<div** with **class=info cast**.

Using list comprehension, we will filter for **<div** tags

In [31]:
cast_list=[x.find('div',class_='info cast') for x in divs]
cast_list[0]

<div class="info cast">
<span class="descriptor">Starring:</span> <a class="" href="/celebrity/gregory_hines/">Gregory Hines</a>, <a class="" href="/celebrity/billy_crystal/">Billy Crystal</a>, <a class="" href="/celebrity/jimmy_smits/">Jimmy Smits</a>, <a class="" href="/celebrity/steven_bauer/">Steven Bauer</a></div>

In [32]:
links=cast_list[0].find_all('a')
links

[<a class="" href="/celebrity/gregory_hines/">Gregory Hines</a>,
 <a class="" href="/celebrity/billy_crystal/">Billy Crystal</a>,
 <a class="" href="/celebrity/jimmy_smits/">Jimmy Smits</a>,
 <a class="" href="/celebrity/steven_bauer/">Steven Bauer</a>]

From the above output, we see the first element of the list has sublists, which means the cast is actally a list of lists. for having a better understanding and view, we will convert a list of lists into a single list.

In [33]:
cast_names=', '.join([x.string for x in links])
cast_names

'Gregory Hines, Billy Crystal, Jimmy Smits, Steven Bauer'

Now we will apply the same logic for all the elements of the cast_list variable

We can do it in two ways. one is doing it with a regular written for loops.

In [34]:
cast=[]

for c in cast_list:
    links=c.find_all('a')
    cast_names=[link.string for link in links]
    result=', '.join(cast_names)
    cast.append(result)
cast[:10]

['Gregory Hines, Billy Crystal, Jimmy Smits, Steven Bauer',
 'Christian Bale, Emily Watson, Taye Diggs, Angus Macfadyen',
 'Jet Li, Tony Leung Chiu Wai, Maggie Cheung, Daoming Chen',
 'Patrick Swayze, Kelly Lynch, Sam Elliott, Ben Gazzara',
 'Denzel Washington, Chris Pine, Rosario Dawson, Ethan Suplee',
 'Richard Roundtree, Moses Gunn, Gwen Mitchell, Christopher St. John',
 'Ok-bin Kim, Kim Seo-hyung, Shin Ha-kyun, Bang Sung-jun',
 'Christopher Lambert, Sean Connery, Roxanne Hart, Clancy Brown',
 'Bruce Willis, Bonnie Bedelia, William Atherton, Reginald VelJohnson',
 'Nicolas Cage, Diane Kruger, Justin Bartha, Sean Bean']

Next is converting the above logic into a list comprehension

In [35]:
cast=[', '.join([link.string for link in c.find_all('a')]) for c in cast_list]
cast[:10]

['Gregory Hines, Billy Crystal, Jimmy Smits, Steven Bauer',
 'Christian Bale, Emily Watson, Taye Diggs, Angus Macfadyen',
 'Jet Li, Tony Leung Chiu Wai, Maggie Cheung, Daoming Chen',
 'Patrick Swayze, Kelly Lynch, Sam Elliott, Ben Gazzara',
 'Denzel Washington, Chris Pine, Rosario Dawson, Ethan Suplee',
 'Richard Roundtree, Moses Gunn, Gwen Mitchell, Christopher St. John',
 'Ok-bin Kim, Kim Seo-hyung, Shin Ha-kyun, Bang Sung-jun',
 'Christopher Lambert, Sean Connery, Roxanne Hart, Clancy Brown',
 'Bruce Willis, Bonnie Bedelia, William Atherton, Reginald VelJohnson',
 'Nicolas Cage, Diane Kruger, Justin Bartha, Sean Bean']

Now left are adjusted scores and Synopsis of the movies.

For adjusted scores , after inspecting the website we find that the cast details are available in **<div** tag with **class=info countdown-adjusted-score**.

Using list comprehension, we will filter for **<div** tags

In [36]:
adjusted_score=[div.find('div', class_='info countdown-adjusted-score') for div in divs]
adjusted_score[0]

<div class="info countdown-adjusted-score"><span class="descriptor">Adjusted Score: </span>61.182% <span class="glyphicon glyphicon-question-sign" data-html="true" data-original-title="The Adjusted Score comes from a weighted formula (Bayesian) that we use that accounts for variation in the number of reviews per movie." data-placement="top" data-toggle="tooltip" rel="tooltip" title=""></span></div>

In [37]:
for x in adjusted_score[0]:
    print(x)

<span class="descriptor">Adjusted Score: </span>
61.182% 
<span class="glyphicon glyphicon-question-sign" data-html="true" data-original-title="The Adjusted Score comes from a weighted formula (Bayesian) that we use that accounts for variation in the number of reviews per movie." data-placement="top" data-toggle="tooltip" rel="tooltip" title=""></span>


Using a for loop we find the contents of the 1st element of the adjusted_stores, and we see that the score is the second element

In [38]:
adjusted_score[0].contents[1]

'61.182% '

We also notice that there's a **%** symbol in the result, and the data type is a string and we need it in integer format. 

So using a list comprehension and the above technique, we will apply the same logic for all the elements of the list.

In [39]:
ad_scores=[score.contents[1].strip('% ') for score in adjusted_score]
ad_scores=[float(scores) for scores in ad_scores]
ad_scores[0:5]

[61.182, 41.992, 100.762, 41.986, 91.476]

The last data we need is the synopsis of each movie. And once again upon inspecting the website html page using the developer tools we will find out that synopsis is in **class=info synopsis** with a **<div** tag.

using find_all method we will find part of the html with **Div** tag and the class.

In [40]:
synopsis=[div.find('div', class_='info synopsis') for div in divs]
synopsis[0]

<div class="info synopsis"><span class="descriptor">Synopsis:</span> Distinguished by a sharp, witty dialogue between its two cop protagonists, Ray and Danny (Gregory Hines and Billy Crystal), this...<a class="" data-pageheader="" href="https://www.rottentomatoes.com/m/1018009-running_scared/" target="_top"> [More]</a></div>

In [41]:
for x in synopsis[0]:
    print(x)

<span class="descriptor">Synopsis:</span>
 Distinguished by a sharp, witty dialogue between its two cop protagonists, Ray and Danny (Gregory Hines and Billy Crystal), this...
<a class="" data-pageheader="" href="https://www.rottentomatoes.com/m/1018009-running_scared/" target="_top"> [More]</a>


And again our movie synopsis is the second part of the content.  and there's an unwanted space at the begining of the description. we will remove it using the strip method.

In [42]:
synopsis[0].contents[1].strip()

'Distinguished by a sharp, witty dialogue between its two cop protagonists, Ray and Danny (Gregory Hines and Billy Crystal), this...'

In [43]:
synopsis_text=[x.contents[1].strip() for x in synopsis]
synopsis_text[0:5]

['Distinguished by a sharp, witty dialogue between its two cop protagonists, Ray and Danny (Gregory Hines and Billy Crystal), this...',
 'In the nation of Libria, there is always peace among men. The rules of the Librian system are simple. If...',
 "Hero is two-time Academy Award nominee Zhang Yimou's directorial attempt at exploring the concept of a Chinese hero. During the...",
 'Dalton (Swayze) is a true gentleman with a degree in philosophy from NYU. He also has a flip side -...',
 'In this action thriller from director Tony Scott, rookie train operator Will (Chris Pine) and grizzled veteran engineer Frank (Denzel...']

### Step -3

We succesfully scraped the rotten tomatoes website for the top action movies in the page, and got all the required data we need. Now we need to create a data frame, which is used to visualize the data in tabluar format.

Other option is to connect to a database and load them into a table in the data base. 

Using the pandas library, we create a dataframe.

In [44]:
movies_info = pd.DataFrame()

movies_info['Movie Title']=movie_names
movies_info['Year']=years
movies_info['scores']=original_scores
movies_info['Adjusted Scores']=ad_scores
movies_info['Director']=directors
movies_info['Synopsis']=synopsis_text
movies_info['cast']=cast
movies_info['Consensus']=consensus_text

movies_info

,Movie Title,Year,scores,Adjusted Scores,Director,Synopsis,cast,Consensus
0,Running Scared,1986,60,61.182,Peter Hyams,"Distinguished by a sharp, witty dialogue betwe...","Gregory Hines, Billy Crystal, Jimmy Smits, Ste...",Running Scared struggles to strike a consisten...
1,Equilibrium,2002,40,41.992,Kurt Wimmer,"In the nation of Libria, there is always peace...","Christian Bale, Emily Watson, Taye Diggs, Angu...",Equilibrium is a reheated mishmash of other sc...
2,Hero,2004,95,100.762,Zhang Yimou,Hero is two-time Academy Award nominee Zhang Y...,"Jet Li, Tony Leung Chiu Wai, Maggie Cheung, Da...",With death-defying action sequences and epic h...
3,Road House,1989,39,41.986,Rowdy Herrington,Dalton (Swayze) is a true gentleman with a deg...,"Patrick Swayze, Kelly Lynch, Sam Elliott, Ben ...",Whether Road House is simply bad or so bad it'...
4,Unstoppable,2010,86,91.476,Tony Scott,In this action thriller from director Tony Sco...,"Denzel Washington, Chris Pine, Rosario Dawson,...","As fast, loud, and relentless as the train at ..."
5,Shaft,1971,88,92.024,Gordon Parks,"Shaft, a highly successful film, spawned an in...","Richard Roundtree, Moses Gunn, Gwen Mitchell, ...",This is the man that would risk his neck for h...
6,The Villainess (Ak-Nyeo),2017,84,86.913,Jung Byung-gil,"Since she was a little girl, Sook-hee was rais...","Ok-bin Kim, Kim Seo-hyung, Shin Ha-kyun, Bang ...",The Villainess offers enough pure kinetic thri...
7,Highlander,1986,69,71.927,Russell Mulcahy,"Among humans for centuries, an immortal specie...","Christopher Lambert, Sean Connery, Roxanne Har...","People hate Highlander because it's cheesy, bo..."
8,Die Hard 2,1990,68,72.291,Renny Harlin,"""Another basement, another elevator...how can ...","Bruce Willis, Bonnie Bedelia, William Atherton...","It lacks the fresh thrills of its predecessor,..."
9,National Treasure,2004,46,50.848,Jon Turteltaub,Benjamin Franklin Gates is a third generation ...,"Nicolas Cage, Diane Kruger, Justin Bartha, Sea...","National Treasure is no treasure, but it's a f..."


## Conclusion

Here, we performed a form of ETL using web scraping. We **Extracted** data from the web page,  and **transformed** it into meaningful data and finally **Loaded** the data into a Data Frame. 